<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#Creating-a-table-of-all-the-values" data-toc-modified-id="Creating-a-table-of-all-the-values-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a table of all the values</a></span><ul class="toc-item"><li><span><a href="#For-2010-to-2014" data-toc-modified-id="For-2010-to-2014-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>For 2010 to 2014</a></span></li><li><span><a href="#For-2015-to-2019" data-toc-modified-id="For-2015-to-2019-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>For 2015 to 2019</a></span></li><li><span><a href="#S&amp;P-index" data-toc-modified-id="S&amp;P-index-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>S&amp;P index</a></span></li></ul></li><li><span><a href="#Three-types-of-images" data-toc-modified-id="Three-types-of-images-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Three types of images</a></span><ul class="toc-item"><li><span><a href="#Recurrence-plot" data-toc-modified-id="Recurrence-plot-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Recurrence plot</a></span></li><li><span><a href="#Gramian-Angular-Field" data-toc-modified-id="Gramian-Angular-Field-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Gramian Angular Field</a></span></li><li><span><a href="#Markov-Transition-Field" data-toc-modified-id="Markov-Transition-Field-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Markov Transition Field</a></span></li></ul></li><li><span><a href="#3-channels" data-toc-modified-id="3-channels-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3-channels</a></span></li><li><span><a href="#Dataset-other-label" data-toc-modified-id="Dataset-other-label-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Dataset other label</a></span><ul class="toc-item"><li><span><a href="#Training" data-toc-modified-id="Training-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#S&amp;P-index" data-toc-modified-id="S&amp;P-index-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>S&amp;P index</a></span></li></ul></li></ul></div>

# Making the images from the time series

## Import

In [ ]:
from pandas import read_csv, DataFrame, to_datetime, Timestamp
from os import listdir, remove
from os.path import isfile, join
from math import log, cos, acos
import numpy as np
from PIL import Image
import pywt
from matplotlib.image import imsave
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
import pickle
from re import split
import datetime as dt

## Creating a table of all the values

### For 2010 to 2014

In [ ]:
# remove('training/AMCR.csv')
# remove('training/TT.csv')
# remove('training/CFG.csv')
# remove('training/INFO.csv')
# remove('training/ANET.csv')

In [ ]:
train_companies_name = [f[:-4] for f in listdir('training') if isfile(join('training', f))]
train_companies_name = [train_companies_name[i] if i%15 < 8 else None for i in range(len(train_companies_name))]
train_num_companies = len(train_companies_name)

In [ ]:
read_csv('training/' + train_companies_name[0] + '.csv')

In [ ]:
train_stocks_data = [0 for x in range(train_num_companies)]

for i in range(train_num_companies):
    if train_companies_name[i] != None:
    
        data = read_csv('training/' + train_companies_name[i] + '.csv')

        opening = [log(x) for x in (data['Open'].values[1:]/data['Open'].values[:-1])][1:]

        close = [log(x) for x in (data['Close'].values[1:]/data['Close'].values[:-1])][:-1]

        high = [log(x) for x in (data['High'].values[1:]/data['High'].values[:-1])][:-1]

        low = [log(x) for x in (data['Low'].values[1:]/data['Low'].values[:-1])][:-1]

        label = [1 if x == True else 0 for x in data['Close'].values > data['Open'].values][2:]

        train_stocks_data[i] = np.array([opening, close, high, low, label])

### For 2015 to 2019

In [ ]:
# remove('testing/AMCR.csv')
# remove('testing/TT.csv')

In [ ]:
test_companies_name = [f[:-4] for f in listdir('testing') if isfile(join('testing', f))]
test_companies_name = [test_companies_name[i] for i in range(0, len(test_companies_name), 5)]

test_num_companies = len(test_companies_name)

In [ ]:
test_stocks_data = []
names = []

for i in range(test_num_companies):
    data = read_csv('testing/' + test_companies_name[i] + '.csv')
    
    if len(data) == 1258:        
        opening = np.array([log(x) for x in (data['Open'].values[1:]/data['Open'].values[:-1])][1:-1])
        
        close = np.array([log(x) for x in (data['Close'].values[1:]/data['Close'].values[:-1])][:-2])
        
        high = np.array([log(x) for x in (data['High'].values[1:]/data['High'].values[:-1])][:-2])
        
        low = np.array([log(x) for x in (data['Low'].values[1:]/data['Low'].values[:-1])][:-2])
        
        date = np.array(to_datetime(data['Date']).sub(Timestamp('2015-01-01')).dt.days.values[2:-1])
        
        label = np.array([1 if x == True else 0 for x in (data['Close'].values > data['Open'])][2:-1])
        
        test_stocks_data.append(np.array([opening, close, high, low, date, label]))
        
        names.append(test_companies_name[i])

test_stocks_data = np.array(test_stocks_data)

test_companies_name = names

test_num_companies = len(test_companies_name)

### S&P index

In [ ]:
data = read_csv('s&p.csv')

opening = [log(x) for x in (data['Open'].values[1:]/data['Open'].values[:-1])][1:]

close = [log(x) for x in (data['Close'].values[1:]/data['Close'].values[:-1])][:-1]

high = [log(x) for x in (data['High'].values[1:]/data['High'].values[:-1])][:-1]

low = [log(x) for x in (data['Low'].values[1:]/data['Low'].values[:-1])][:-1]

close_values = data['Close'].values[2:]

open_values = data['Open'].values[2:]

date = to_datetime(data['Date']).sub(Timestamp('2015-01-01')).dt.days.values[2:]

sandp = np.array([opening, close, high, low, close_values, open_values])

## Three types of images

### Recurrence plot

In [ ]:
def rp(table, show = False):
    n = len(table)
    
    #First, we rescale (a copy of) our table from 0 to 1
    maxi = max(table)
    mini = min(table)
    
    #If all the elements of the table are equal, we only want a black image 
    if (maxi == mini):
        img = 255 * np.ones((n,n))
        
    else: 
        copy_table = table.copy()
        for i in range(n):
            copy_table[i] = (copy_table[i] - mini)/ (maxi - mini)

        #Now, we create our recurrence plot image
        img = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                img[i][j] = 1 - (1 if (abs(copy_table[i] - copy_table[j]) < 0.1) else (abs(copy_table[i] - copy_table[j])))

        #We need to rescale our values from 0 to 255, to obtain a greyscale
        mini = np.amin(img)
        maxi = np.amax(img)
        for i in range(n):
            for j in range(n):
                img[i][j] = int((img[i][j] - mini)/ (maxi - mini) * 255)
    
    if show: 
        imshow(img, cmap = 'gray')
    
    return img

In [ ]:
imshow(rp(train_stocks_data[0][0][240:256]), cmap='gray')

### Gramian Angular Field

In [ ]:
def gaf(table, show = False):
    n = len(table)
    
    #First, we rescale (a copy of) our table from -1 to 1
    maxi = max(table)
    mini = min(table)
    
    #If all the elements of the table are equal, we only want a black image 
    if (maxi == mini):
        img = 255 * np.ones((n,n))
        
    else: 
        copy_table = table.copy()
        for i in range(n):
            copy_table[i] = (copy_table[i] - mini)/ (maxi - mini)
            if abs(copy_table[i]) > 1:
                copy_table[i] = np.sign(copy_table[i])
            copy_table[i] = acos(copy_table[i])

        #Now, we create our recurrence plot image
        img = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                img[i][j] = cos(copy_table[i] + copy_table[j])

        #We need to rescale our values from 0 to 255, to obtain a greyscale
        for i in range(n):
            for j in range(n):
                img[i][j] = int((img[i][j] + 1) * 127.5)
    
    if show: 
        imshow(img, cmap = 'grey')
    
    return img

In [ ]:
imshow(gaf(train_stocks_data[0][0][240:256]), cmap='gray')

### Markov Transition Field

In [ ]:
Q = 16

In [ ]:
def quantile(arr, nb):
    thresh = [np.quantile(arr, (i + 1.0) / nb) for i in range(nb)]
    res = np.zeros(len(arr))
    for i in range(len(arr)):
        quantile = 0
        while arr[i] > thresh[quantile]:
            quantile += 1
        res[i] = quantile
    return res

In [ ]:
def mtf(arr, show = False, nb = Q):
    
    quant = quantile(arr, nb)
    img = np.zeros((nb,nb))
    
    for i in range(len(quant) - 1):
        abscissa = int(quant[i])
        ordinate = int(quant[i + 1])
        img[abscissa, ordinate] += 1
    
    for i in range(nb):
        if sum(img[:,i]) != 0:
            img[:,i] /= sum(img[:,i])
        
    img = (-img + 1) * 255
    
    if show: 
        imshow(img, cmap = 'grey')
    
    return img

In [ ]:
imshow(mtf(train_stocks_data[0][0][:256]), cmap='gray')

## 3-channels

In [ ]:
def three_channels(table_open, table_close, table_high, table_low, table_open_mtf, table_close_mtf, table_high_mtf, table_low_mtf, show = False):
    
    ### THIS IS THE MTF PART OF THE IMAGE ###
    
    mtf_open = mtf(table_open_mtf)
    mtf_close = mtf(table_close_mtf)
    mtf_high = mtf(table_high_mtf)
    mtf_low = mtf(table_low_mtf)
    
    mtf_img = np.concatenate((np.concatenate((mtf_open,mtf_high), axis = 1), 
                            np.concatenate((mtf_low,mtf_close), axis = 1)), 
                            axis = 0)
    
        
    ### THIS IS THE GAF PART OF THE IMAGE ###
    
    gaf_open = gaf(table_open)
    gaf_close = gaf(table_close)
    gaf_high = gaf(table_high)
    gaf_low = gaf(table_low)
    
    gaf_img = np.concatenate((np.concatenate((gaf_open,gaf_high), axis = 1), 
                            np.concatenate((gaf_low,gaf_close), axis = 1)), 
                            axis = 0)
    
    
    ### THIS IS THE RP PART OF THE IMAGE ###
    
    rp_open = rp(table_open)
    rp_close = rp(table_close)
    rp_high = rp(table_high)
    rp_low = rp(table_low)
    
    for x in [rp_open, rp_close, rp_high, rp_low]:
        LL, (LH, HL, HH) = pywt.dwt2(x, 'bior1.3')
        x = np.concatenate((np.concatenate((LL,LH), axis = 1), 
                            np.concatenate((HL,HH), axis = 1)), 
                            axis = 0)
    
        #We need to rescale our values from 0 to 255, to obtain a greyscale
        mini = np.amin(x)
        maxi = np.amax(x)
        for i in range(len(x)):
            for j in range(len(x)):
                x[i][j] = int((x[i][j] - mini)/ (maxi - mini) * 255)
    
    wt_img = np.concatenate((np.concatenate((rp_open,rp_high), axis = 1), 
                            np.concatenate((rp_low,rp_close), axis = 1)), 
                            axis = 0)
    
    
    
    ### THIS IS THE CONCATENATE OF THE IMAGE ###

    img = np.dstack((np.array(gaf_img, dtype='uint8'), np.array(wt_img, dtype='uint8'), np.array(mtf_img, dtype='uint8')))
    
    if show:
        imshow(img)
    
    return img

In [ ]:
imshow(three_channels(train_stocks_data[3][0][240:256], 
               train_stocks_data[3][1][240:256], 
               train_stocks_data[3][2][240:256], 
               train_stocks_data[3][3][240:256],
               train_stocks_data[3][0][:256], 
               train_stocks_data[3][1][:256], 
               train_stocks_data[3][2][:256], 
               train_stocks_data[3][3][:256]))

## Dataset other label

### Training

In [ ]:
train_size = 0
for i in range(train_num_companies):
    for j in range(len(train_stocks_data[i][0]) - 256):
        train_size += 1

In [ ]:
train_images = [0 for i in range(train_size)]
train_labels = [0 for i in range(train_size)]
compt = 0

for i in range(train_num_companies):
    if i%10==0:
        print(str(i) + ' / ' + str(train_num_companies))
    for j in range(240, len(train_stocks_data[i][0]) - 16):
        train_images[compt] = three_channels(train_stocks_data[i][0][j : j + 16],
                                             train_stocks_data[i][1][j : j + 16], 
                                             train_stocks_data[i][2][j : j + 16],
                                             train_stocks_data[i][3][j : j + 16],
                                             train_stocks_data[i][0][j - 240 : j + 16],
                                             train_stocks_data[i][1][j - 240 : j + 16], 
                                             train_stocks_data[i][2][j - 240 : j + 16],
                                             train_stocks_data[i][3][j - 240 : j + 16])
        
        train_labels[compt] = (1 if (train_stocks_data[i][0][j + 15] < train_stocks_data[i][1][j + 16]) else 0)
        
        compt += 1

In [ ]:
train_labels[218514]

In [ ]:
with open('objs_train_label.pkl', 'wb') as f: 
    pickle.dump([train_images[:218515], train_labels[:218515]], f)

### Testing

In [ ]:
test_days = len(test_stocks_data[0][0]) - 15

for j in range(240, test_days):
    if j%10 == 0:
        print(str(j) + '/' + str(test_days))
        
    images = [0 for i in range(test_num_companies)]
    labels = [0 for i in range(test_num_companies)]
    
    for i in range(test_num_companies):
            images[i] = three_channels(test_stocks_data[i][0][j : j + 16],
                                       test_stocks_data[i][1][j : j + 16], 
                                       test_stocks_data[i][2][j : j + 16],
                                       test_stocks_data[i][3][j : j + 16],
                                       test_stocks_data[i][0][j - 240 : j + 16],
                                       test_stocks_data[i][1][j - 240 : j + 16], 
                                       test_stocks_data[i][2][j - 240 : j + 16],
                                       test_stocks_data[i][3][j - 240 : j + 16])

            labels[i] = (1 if (test_stocks_data[i][0][j + 15] < test_stocks_data[i][1][j + 16]) else 0)

    with open('testing_label/objs_' + str(j) + '.pkl', 'wb') as f: 
        pickle.dump([images, labels], f, protocol=-1)

### S&P index

In [ ]:
num_samples = len(sandp[0]) - 15

sp_images = [0 for i in range(num_samples)]
sp_values = [0 for i in range(num_samples)]

for i in range(240, num_samples):
    if i%10==0:
        print(str(i) + '/' + str(num_samples))
    sp_images[i] = three_channels(sandp[0][i : i + 16],
                                  sandp[1][i : i + 16], 
                                  sandp[2][i : i + 16],
                                  sandp[3][i : i + 16],
                                  sandp[0][i - 240 : i + 16],
                                  sandp[1][i - 240 : i + 16], 
                                  sandp[2][i - 240 : i + 16],
                                  sandp[3][i - 240 : i + 16])

    sp_values[i] = sandp[4][i + 15] / sandp[5][i + 15]


In [ ]:
with open('objs_sandp_label.pkl', 'wb') as f: 
    pickle.dump([sp_images[240:], sp_values[240:]], f, protocol=-1)